In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from webdriver_manager.chrome import ChromeDriverManager 
import pandas as pd
from time import sleep

In [2]:
range_low = '35078000'
range_high = '35080000'
start_date = '01/08/2020'
end_date = '31/08/2020'

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.hacienda.go.cr/tica/web/hcitrncm1.aspx')

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/home/andres/.wdm/drivers/chromedriver/linux64/85.0.4183.87/chromedriver] found in cache


In [4]:
img_src = driver.find_element_by_xpath('//div[@id="captchaImage"]/img')


In [5]:
img_src = img_src.get_attribute('src')
captchas = pd.read_csv('../Captchas/captchas.csv', index_col=0)
captcha_number = int(img_src.split('/')[-1].split('.')[0])
captcha_value = captchas.iloc[captcha_number - 1]
captcha_value

 captcha_answer    jump
Name: 75, dtype: object

In [6]:
driver.find_element_by_xpath('//select[@id="vVADUANA"]/option[@value="X"]').click()

In [7]:
date_i = driver.find_element_by_xpath('//div[@id="vVFCHIN_dp_container"]/input')

In [8]:
date_i.send_keys(start_date)

In [9]:
date_f = driver.find_element_by_xpath('//div[@id="vVFCHFN_dp_container"]/input')

In [10]:
date_f.send_keys(end_date)

In [11]:
range_start = driver.find_element_by_xpath('//td/input[@id="vVNCMINI"]')
range_start.clear()
range_start.send_keys(range_low)

In [12]:
range_end = driver.find_element_by_xpath('//td/input[@id="vVNCMFIN"]')
range_end.clear()
range_end.send_keys(range_high)

In [13]:
captcha_field = driver.find_element_by_xpath('//input[@id="_cfield"]')

In [14]:
captcha_field.clear()

In [15]:
captcha_field.send_keys(captcha_value)

In [16]:
driver.find_element_by_xpath('//input[@name="BUTTON1"]').click()

In [17]:
# Lists to store scraped data points
main_pg_dfs = []
duas_dfs = []
declarantes = []
tipos_cambio = []
bultos = []
razon_social = []
facturas_dfs = []

In [18]:
scrape = True

while scrape:
### Navigates Through Result Pages
    print('New Page')
    sleep(1)
    source = driver.page_source

    tables = pd.read_html(source)

    len_test = tables[9]
    
    sleep(1)

    scrape_pg = True
    
    while scrape_pg:
    ### Opens Every Result Link Withing the Page
        sleep(1)
        source = driver.page_source

        tables = pd.read_html(source)

        main_pg_data = tables[9]
        main_pg_dfs.append(main_pg_data)
    
        duas_df = main_pg_data.groupby('Numero', sort=False).size()
        duas_dfs.append(duas_df)
        
        for i in range(1, len(duas_df) +1):
            sleep(2)
            line = duas_df.iloc[:i].sum()
            if (line != 10 and len(main_pg_data)//10 !=0) or (len(main_pg_data)//10 == 0):
                xpath = f'//table[@id="SubfileContainerTbl"]/tbody/tr/td[@colindex="2"]/span[@id="span_vVNUMECORRE_000{str(line)}"]/a'
                sleep(2)
                driver.find_element_by_xpath(xpath).click()
                sleep(2)
                iframe = driver.find_element_by_xpath('//iframe')
                driver.switch_to.frame(iframe)
                sleep(1)
                
                ### Scraping Main iFrame
                declarante = driver.find_element_by_xpath('//span[@id="span_vVNOMDECL"]').text
                declarantes.append(declarante)
                tipo_cambio_v = float(driver.find_element_by_xpath('//span[@id="span_TIPCAMVEN"]').text)
                tipos_cambio.append(tipo_cambio_v)
                n_lineas = int(driver.find_element_by_xpath('//span[@id="span_vVLINEAS"]').text)
                
                ### Lineas Dua iFrame
                lineas_dfs = []
                pgs = n_lineas // 20 + 1
                driver.find_element_by_xpath('//input[@name="LINEAS"]').click()
                sleep(2)
                for page in range(pgs):
                ### Iterating over pages within this section of iframe
                    sleep(1)
    
                    lineas_src = driver.page_source
                    lineas_dfs.append(pd.read_html(lineas_src))
    
                    driver.find_element_by_xpath('//input[@name="SIGUIENTE"]').click()
                lineas_dfs = [lineas_dfs[i][1] for i in range(pgs)]
                lineas_df = pd.concat(lineas_dfs)
                lineas_df[['Linea', 'Partida', 'Bultos']].astype('int').dtypes
                lineas_df['Partida'] = lineas_df['Partida'].apply(lambda x: int(x/10000))
                lineas_df = lineas_df[(lineas_df['Partida'] > int(range_low)) & (lineas_df['Partida'] < int(range_high))]
                n_bultos = lineas_df['Bultos'].sum()
                bultos.append(n_bultos)
                sleep(2)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(2)
                
                ### Manifesto/Stock iFrame
                driver.find_element_by_xpath('//input[@name="AER"]').click()
                sleep(1)
                try:
                    driver.find_element_by_xpath('//span[@id="span_CGMOVSKID_0001"]').click()
                    sa = driver.find_element_by_xpath('//span[@id="span_vVCONSNOM_0001"]').text
                    ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                    sleep(1)
                except:
                    sa = 0 
                razon_social.append(sa)
                
                sleep(1)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(2)
                
                ### Facturas iFrame
                driver.find_element_by_xpath('//input[@name="FACTURAS"]').click()
                sleep(2)
                fact_src = driver.page_source
                fact_dfs = pd.read_html(fact_src)
                lines_of_interest = lineas_df['Linea'].values
                fact_dfs = fact_dfs[4]
                fact_dfs['Lin.Dua'].astype('int').dtypes
                fact_dfs[['Cant. prod. Und comerc', 'Precio unid. mercancía']].astype('float').dtypes
                fact_dfs = fact_dfs.set_index('Lin.Dua').loc[lines_of_interest]
                facturas_dfs.append(fact_dfs)
                
                sleep(1)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(1)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(1)
                if len(main_pg_data)//10 == 0:
                    scrape_pg = False
            
            
            elif line == 10:
                xpath = f'//table[@id="SubfileContainerTbl"]/tbody/tr/td[@colindex="2"]/span[@id="span_vVNUMECORRE_00{str(line)}"]/a'
                sleep(2)
                driver.find_element_by_xpath(xpath).click()
                sleep(2)
                iframe = driver.find_element_by_xpath('//iframe')
                driver.switch_to.frame(iframe)
                sleep(1)
                
                ### Scraping Main iFrame
                declarante = driver.find_element_by_xpath('//span[@id="span_vVNOMDECL"]').text
                declarantes.append(declarante)
                tipo_cambio_v = float(driver.find_element_by_xpath('//span[@id="span_TIPCAMVEN"]').text)
                tipos_cambio.append(tipo_cambio_v)
                n_lineas = int(driver.find_element_by_xpath('//span[@id="span_vVLINEAS"]').text)
                
                ### Lineas Dua iFrame
                lineas_dfs = []
                pgs = n_lineas // 20 + 1
                driver.find_element_by_xpath('//input[@name="LINEAS"]').click()
                sleep(2)
                for page in range(pgs):
                ### Iterating over pages within this section of iframe
                    sleep(1)
    
                    lineas_src = driver.page_source
                    lineas_dfs.append(pd.read_html(lineas_src))
    
                    driver.find_element_by_xpath('//input[@name="SIGUIENTE"]').click()
                lineas_dfs = [lineas_dfs[i][1] for i in range(pgs)]
                lineas_df = pd.concat(lineas_dfs)
                lineas_df[['Linea', 'Partida', 'Bultos']].astype('int').dtypes
                lineas_df['Partida'] = lineas_df['Partida'].apply(lambda x: int(x/10000))
                lineas_df = lineas_df[(lineas_df['Partida'] > int(range_low)) & (lineas_df['Partida'] < int(range_high))]
                n_bultos = lineas_df['Bultos'].sum()
                bultos.append(n_bultos)
                sleep(2)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(2)
                
                ### Manifesto/Stock iFrame
                driver.find_element_by_xpath('//input[@name="AER"]').click()
                sleep(1)
                try:
                    driver.find_element_by_xpath('//span[@id="span_CGMOVSKID_0001"]').click()
                    sa = driver.find_element_by_xpath('//span[@id="span_vVCONSNOM_0001"]').text
                    ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                    sleep(1)
                except:
                    sa = 0 
                razon_social.append(sa)
                
                sleep(1)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(2)
                
                ### Facturas iFrame
                driver.find_element_by_xpath('//input[@name="FACTURAS"]').click()
                sleep(2)
                fact_src = driver.page_source
                fact_dfs = pd.read_html(fact_src)
                lines_of_interest = lineas_df['Linea'].values
                fact_dfs = fact_dfs[4]
                fact_dfs['Lin.Dua'].astype('int').dtypes
                fact_dfs[['Cant. prod. Und comerc', 'Precio unid. mercancía']].astype('float').dtypes
                fact_dfs = fact_dfs.set_index('Lin.Dua').loc[lines_of_interest]
                facturas_dfs.append(fact_dfs)
                sleep(2)
                
                sleep(1)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(1)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(1)
                scrape_pg = False
    
    
    
    if len(len_test)//10 != 0:
        sleep(1)
        driver.find_element_by_xpath('//input[@title="Siguiente"]').click()
        sleep(1)
    else:
        print('Job Done!')
        scrape = False

New Page
New Page
New Page
New Page
New Page
New Page
Job Done!


In [19]:
driver.quit()

In [20]:
len(main_pg_dfs)

6

In [21]:
len(duas_dfs)

6

In [22]:
len(declarantes)

36

In [23]:
len(tipos_cambio)

36

In [24]:
len(bultos)

36

In [25]:
len(razon_social)

36

In [27]:
len(facturas_dfs)

36

In [40]:
main_pg_results = pd.concat(main_pg_dfs).reset_index().drop('index', axis=1)

In [42]:
main_pg_results.to_csv('Agosto_Resultados_Pag_Principal.csv', index=False)

In [44]:
duas_data = pd.concat(duas_dfs)

In [45]:
len(duas_data)

36

In [49]:
data = pd.DataFrame(duas_data, columns={'Cantidad'})

In [53]:
data['Declarante'] = declarantes
data['Tipo Cambio'] = tipos_cambio
data['Bultos'] = bultos
data['Razon Social'] = razon_social

In [56]:
data

,Cantidad,Declarante,Tipo Cambio,Bultos,Razon Social
Numero,,,,,
45543,3,MUDANZAS MUNDIALES SOCIEDAD ANONIMA,599.13,10,BIMBO DE COSTA RICA SOCIEDAD ANONIMA
45835,1,GRUPO SERVICA COSTA RICA SOCIEDAD ANONIMA,600.11,5,JAVA CENTROAMERICANA AIME SOCIEDAD ANONIMA
45987,1,MUDANZAS MUNDIALES SOCIEDAD ANONIMA,598.23,4,BIMBO DE COSTA RICA SOCIEDAD ANONIMA
55273,1,RODRIGUEZ MURILLO VLADIMIR,595.91,0,0
360929,3,AGENCIA ADUANAL SIETE MARES SOCIEDAD ANONIMA,592.22,88,FARYVET SOCIEDAD ANONIMA
360964,1,ARAYA CHAVARRIA BORIS GERARDO,592.22,0,ASESORIA EN ALIMENTOS ALFA SOCIEDAD ANONIMA
361809,2,SERVICIOS INTERNACIONALES DE CARGA SICSA SOCIE...,594.44,1,FRUCTA CR SOCIEDAD ANONIMA
362325,1,AGENCIA ADUANAL SIETE MARES SOCIEDAD ANONIMA,594.44,1,CARTASA DE SOL SOCIEDAD ANONIMA
364524,1,ALFARO GUTIERREZ MARLON ALBERTO,595.21,4,ALIMENTOS LIGEROS DE CENTRO AMERICA SOCIEDAD A...


In [78]:
temp = [facturas_dfs[i][['Cant. prod. Und comerc', 'Precio unid. mercancía']].sum() for i in range(len(facturas_dfs))]

In [79]:
weights = [temp[i][0] for i in range(len(temp))]

In [82]:
data['Peso (kg)'] = weights

In [83]:
cost = [temp[i][1] for i in range(len(temp))]

In [84]:
data['Precio Unidad Mercancía ($)'] = cost

In [86]:
data['Total ($)'] = data['Peso (kg)']*data['Precio Unidad Mercancía ($)']

In [88]:
data.to_csv('Agosto_Data.csv', index=True)

In [96]:
writer = pd.ExcelWriter('Datos_Agosto.xlsx', engine='xlsxwriter')

In [97]:
data.to_excel(writer, sheet_name='Detalle', index=True)

In [98]:
main_pg_results.to_excel(writer, sheet_name='Descripcion', index=False)

In [99]:
writer.save()